In [ ]:
from ultralytics import YOLO
import torch
import os
import fnmatch
import numpy as np
from sklearn import metrics


Ultralytics 8.3.39  Python-3.11.9 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
YOLO11n-cls summary (fused): 112 layers, 1,528,586 parameters, 0 gradients, 3.2 GFLOPs
image 1/2 C:\CS4487\.venv\Lib\site-packages\ultralytics\assets\bus.jpg: 224x224 0_real 0.59, 1_fake 0.41, 13.1ms
image 2/2 C:\CS4487\.venv\Lib\site-packages\ultralytics\assets\zidane.jpg: 224x224 0_real 1.00, 1_fake 0.00, 13.1ms
Speed: 21.0ms preprocess, 13.1ms inference, 0.0ms postprocess per image at shape (2, 3, 224, 224)
Results saved to C:\Underwatermelon\runs\detect\train26


In [ ]:
def load_path(path):
    img_paths = []
    for root, dirs, files in os.walk(path):
        for name in files:
            if fnmatch.fnmatch(name, "*.jpg"):
                label = 0 if '0_real' in root else 1
                img_paths.append((os.path.join(root, name), label))
    return img_paths

In [ ]:
def evaluate(model, test_loader):
    # Validation phase
    model.eval()
    
    y_true = []
    y_pred = []
    
    for img, label in test_loader:
        # Please make sure that the "pred" is binary result
        results = model(img, verbose=False)
        for result in results:
            pred = result.probs.top1
               
        y_true.append(label)
        y_pred.append(pred)

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    accuracy = metrics.accuracy_score(y_true, y_pred)
    return accuracy

In [ ]:
model = YOLO(r"C:\CS4487\yololog\weights\best.pt",task='classify',verbose=False)  # load a pretrained model (recommended for training)
path = "AIGC-Detection-Dataset/AIGC-Detection-Dataset/test"
img_paths = load_path(path)

accuracy = evaluate(model, img_paths)
print(f"Accuracy: {accuracy}")


image 1/58 c:\CS4487\nn\..\AIGC-Detection-Dataset\AIGC-Detection-Dataset\temptest\1_fake\Designer (10).jpeg: 224x224 0_real 1.00, 1_fake 0.00, 6.4ms
image 2/58 c:\CS4487\nn\..\AIGC-Detection-Dataset\AIGC-Detection-Dataset\temptest\1_fake\Designer (11).jpeg: 224x224 0_real 1.00, 1_fake 0.00, 5.0ms
image 3/58 c:\CS4487\nn\..\AIGC-Detection-Dataset\AIGC-Detection-Dataset\temptest\1_fake\Designer (12).jpeg: 224x224 0_real 1.00, 1_fake 0.00, 5.0ms
image 4/58 c:\CS4487\nn\..\AIGC-Detection-Dataset\AIGC-Detection-Dataset\temptest\1_fake\Designer (13).jpeg: 224x224 0_real 1.00, 1_fake 0.00, 5.0ms
image 5/58 c:\CS4487\nn\..\AIGC-Detection-Dataset\AIGC-Detection-Dataset\temptest\1_fake\Designer (14).jpeg: 224x224 0_real 1.00, 1_fake 0.00, 6.0ms
image 6/58 c:\CS4487\nn\..\AIGC-Detection-Dataset\AIGC-Detection-Dataset\temptest\1_fake\Designer (15).jpeg: 224x224 0_real 1.00, 1_fake 0.00, 6.0ms
image 7/58 c:\CS4487\nn\..\AIGC-Detection-Dataset\AIGC-Detection-Dataset\temptest\1_fake\Designer (16).jp

[ultralytics.engine.results.Results object with attributes:
 
 boxes: None
 keypoints: None
 masks: None
 names: {0: '0_real', 1: '1_fake'}
 obb: None
 orig_img: array([[[158, 133,   0],
         [218, 193,  59],
         [207, 183,  47],
         ...,
         [215, 216,  58],
         [221, 215,  66],
         [222, 214,  70]],
 
        [[144, 119,   0],
         [172, 147,  13],
         [164, 139,   5],
         ...,
         [211, 212,  49],
         [212, 207,  55],
         [212, 205,  58]],
 
        [[182, 157,  25],
         [155, 130,   0],
         [187, 162,  28],
         ...,
         [206, 206,  38],
         [210, 205,  48],
         [213, 207,  54]],
 
        ...,
 
        [[ 51,  29,   4],
         [ 51,  32,   5],
         [ 52,  33,   6],
         ...,
         [ 46,  50,  15],
         [ 49,  50,  18],
         [ 48,  52,  17]],
 
        [[ 52,  31,   9],
         [ 49,  31,   8],
         [ 48,  30,   7],
         ...,
         [ 44,  49,  17],
         [ 46,